In [1]:
import wget
import os.path as osp

# Check if disprot data is already downloaded:
if not osp.exists('disprot.tsv'):
    wget.download("https://disprot.org/api/search?release=2022_03&show_ambiguous=true&show_obsolete=false&format=tsv&namespace=all&get_consensus=false", "disprot.tsv")

In [1]:
import pandas as pd
from aaindex.aaindex import aaindex

aa_index_feats = aaindex.record_codes()

disprot = pd.read_csv('disprot.tsv', sep='\t', header=0)
# Drop duplicates:
disprot = disprot.drop_duplicates(subset=['acc', 'start', 'end'])
disprot

,acc,name,organism,ncbi_taxon_id,disprot_id,region_id,start,end,term_namespace,term,ec,reference,region_sequence,confidence,obsolete
0,P03265,DNA-binding protein,Human adenovirus C serotype 5,28285,DP00003,DP00003r002,294,334,Structural state,IDPO:00076,ECO:0006220,pmid:8632448,EHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNT,NaN,NaN
1,P03265,DNA-binding protein,Human adenovirus C serotype 5,28285,DP00003,DP00003r004,454,464,Structural state,IDPO:00076,ECO:0006220,pmid:8632448,VYRNSRAQGGG,NaN,NaN
2,P49913,Cathelicidin antimicrobial peptide,Homo sapiens,9606,DP00004,DP00004r001,134,170,Structural state,IDPO:00076,ECO:0006206,pmid:9452503,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,NaN,NaN
5,P03045,Antitermination protein N,Escherichia phage lambda,10710,DP00005,DP00005r001,1,107,Structural state,IDPO:00076,ECO:0006165,pmid:9659923,MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRPILSLNRKPKS...,NaN,NaN
14,P03045,Antitermination protein N,Escherichia phage lambda,10710,DP00005,DP00005r012,1,22,Structural transition,IDPO:00050,ECO:0006165,pmid:9659923,MDAQTRRRERRAEKQAQWKAAN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10007,A0A2Z5UJ33,Nucleoprotein,Influenza A virus,382835,DP03573,DP03573r005,73,91,Structural state,IDPO:00076,ECO:0006220,pmid:17151603,ERRNKYLEEHPSAGKDPKK,NaN,NaN
10008,A0A2Z5UJ33,Nucleoprotein,Influenza A virus,382835,DP03573,DP03573r006,203,212,Structural state,IDPO:00076,ECO:0006220,pmid:17151603,DRNFWRGENG,NaN,NaN
10010,P03496,Non-structural protein 1,Influenza A virus (strain A/Puerto Rico/8/1934...,211044,DP03575,DP03575r001,204,230,Structural state,IDPO:00076,ECO:0006220,pmid:21464929,RSSNENGRPPLTPKQKREMAGTIRSEV,NaN,NaN
10011,P03496,Non-structural protein 1,Influenza A virus (strain A/Puerto Rico/8/1934...,211044,DP03575,DP03575r002,73,82,Structural state,IDPO:00076,ECO:0006220,pmid:20133840,SDEALKMTMA,NaN,NaN


In [3]:
import requests
import time

def full_seq_from_uniprot(uniprot_id):
    url = 'https://www.uniprot.org/uniprot/' + uniprot_id + '.fasta'
    fasta = requests.get(url).text
    #time.sleep(1)
    return "".join(fasta.split('\n')[1:])

asc2seq = {}
for acc in disprot['acc'].unique():
    asc2seq[acc] = full_seq_from_uniprot(acc)

len(asc2seq)

2365

In [4]:
def get_non_disordered_data(seq, regions):
    for region in sorted(regions, key=lambda x: x[0], reverse=True):
        seq = seq[:region[0]] + seq[region[1]:]
    return seq

asc2non_disordered_seq = {}
for acc in asc2seq:
    start_pos = disprot.loc[disprot['acc'] == acc, 'start'].values
    end_pos = disprot.loc[disprot['acc'] == acc, 'end'].values
    asc2non_disordered_seq[acc] = get_non_disordered_data(asc2seq[acc], list(zip(start_pos, end_pos)))

len(asc2non_disordered_seq)

2365

In [8]:
import numpy as np

def get_avg_feats_per_sequence(seq):
    features = np.zeros(len(aa_index_feats))
    for i, feat in enumerate(aa_index_feats):
        feat_vals = aaindex[feat]['values']
        features[i] = np.average(np.array([feat_vals[aa] for aa in seq.replace('X', '').replace('U', '').replace('Z', '')]))
    return features

prepared_data = []
for asc, ordered_seq in asc2non_disordered_seq.items():
    prepared_data.append([asc, 0, disprot.loc[disprot['acc'] == asc, 'ncbi_taxon_id'].values[0], len(ordered_seq)] + list(get_avg_feats_per_sequence(ordered_seq)))

for i, row in disprot.iterrows():
    prepared_data.append([row['acc'], 1, row['ncbi_taxon_id'], len(row['region_sequence'])] + list(get_avg_feats_per_sequence(row['region_sequence'])))

prepared_data = pd.DataFrame(prepared_data, columns=['asc', 'is_disordered', 'taxon', 'seq_length']+[f'{feat}_avg' for feat in aa_index_feats])

prepared_data.head()

C:\Users\austi\anaconda3\envs\IDP-Predictor\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\austi\anaconda3\envs\IDP-Predictor\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,asc,is_disordered,taxon,seq_length,ANDN920101_avg,ARGP820101_avg,ARGP820102_avg,ARGP820103_avg,AURR980101_avg,AURR980102_avg,...,YUTK870104_avg,ZASB820101_avg,ZHOH040101_avg,ZHOH040102_avg,ZHOH040103_avg,ZIMJ680101_avg,ZIMJ680102_avg,ZIMJ680103_avg,ZIMJ680104_avg,ZIMJ680105_avg
0,P03265,0,28285,479,4.377641,0.904342,0.970501,1.018914,1.021482,1.014948,...,17.175470,-0.156228,2.749415,2.856054,12.326514,1.272234,14.983299,15.802401,6.103758,9.526305
1,P49913,0,9606,134,4.367090,0.887015,1.068507,1.090373,0.998955,0.990000,...,17.045000,-0.152060,2.848507,2.941045,12.744030,1.287090,15.187239,14.644030,6.095075,9.676866
2,P03045,0,10710,1,4.520000,1.180000,2.670000,2.960000,0.880000,1.120000,...,18.490000,-0.107000,3.630000,3.910000,15.700000,1.400000,16.250000,1.430000,5.740000,14.900000
3,P00004,0,9796,1,4.520000,1.180000,2.670000,2.960000,0.880000,1.120000,...,18.490000,-0.107000,3.630000,3.910000,15.700000,1.400000,16.250000,1.430000,5.740000,14.900000
4,P27695,0,9606,193,4.360000,0.944352,1.061969,1.103782,1.015596,1.009275,...,17.097824,-0.138544,2.995026,3.021554,13.261658,1.345751,15.096010,13.535596,6.144145,10.127979


In [9]:
prepared_data.to_csv('basic_aa_features.csv', index=False)

In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd

# TODO: Run linear models to see if any of the features are important
# Then run a linear model on the important ones to see how predictive that is overall
prepared_data = pd.read_csv('basic_aa_features.csv', header=0)
prepared_data = prepared_data.dropna()
prepared_data = prepared_data[prepared_data['seq_length'] > 1]

from scipy.stats import zscore
prepared_data['is_disordered'] = prepared_data['is_disordered'] - 0.5

for col in prepared_data.columns:
    if col != 'asc' and col != 'is_disordered' and col != 'taxon':
        prepared_data[col] = zscore(prepared_data[col])

prepared_data

C:\Users\austi\anaconda3\envs\IDP-Predictor\lib\site-packages\ete3\ncbi_taxonomy\ncbiquery.py:243: UserWarning: taxid 39113 was translated into 2847087
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
C:\Users\austi\anaconda3\envs\IDP-Predictor\lib\site-packages\ete3\ncbi_taxonomy\ncbiquery.py:243: UserWarning: taxid 5145 was translated into 2587412
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))


,asc,is_disordered,taxon,seq_length,ANDN920101_avg,ARGP820101_avg,ARGP820102_avg,ARGP820103_avg,AURR980101_avg,AURR980102_avg,...,YUTK870104_avg,ZASB820101_avg,ZHOH040101_avg,ZHOH040102_avg,ZHOH040103_avg,ZIMJ680101_avg,ZIMJ680102_avg,ZIMJ680103_avg,ZIMJ680104_avg,ZIMJ680105_avg
0,P03265,-0.5,Tectiliviricetes,0.536091,0.278658,0.499579,0.360155,0.342056,-0.020322,-0.088246,...,-0.136091,0.256938,0.297587,0.255449,0.369188,0.440817,0.407614,0.040241,0.161141,0.290198
1,P49913,-0.5,Mammalia,-0.113707,0.067797,0.398724,0.838054,0.711161,-0.486261,-0.750936,...,-0.255985,0.395604,0.612791,0.552614,0.667072,0.504211,0.577280,-0.159101,0.144185,0.407681
4,P27695,-0.5,Mammalia,-0.002582,-0.073883,0.732457,0.806170,0.780424,-0.142074,-0.238943,...,-0.207443,0.845276,1.078856,0.834113,1.036383,0.754541,0.501384,-0.349849,0.240006,0.759686
6,P32774,-0.5,Saccharomycetes,-0.164560,-0.363376,0.366053,0.417411,0.395455,-0.719813,-1.336813,...,0.187128,0.284409,0.857446,0.776395,0.992392,0.452820,0.835924,-0.223451,-0.249148,0.311969
7,P0DMM9,-0.5,Mammalia,0.080291,0.021401,1.096387,0.550908,0.562121,0.146255,-0.119923,...,0.207152,0.629080,1.109442,1.065316,0.939710,0.967313,1.023103,-0.209315,-0.050778,1.033841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7006,A0A2Z5UJ33,0.5,Insthoviricetes,-0.330306,0.809633,0.253737,-1.975164,-1.927782,1.647248,1.667700,...,-0.822589,-0.194273,-0.982968,-0.424526,-1.700006,0.357167,-0.280824,2.419626,1.376814,-1.648227
7007,A0A2Z5UJ33,0.5,Insthoviricetes,-0.347257,1.944532,-0.654877,-2.065758,-1.739113,1.210716,2.144437,...,-2.654438,0.617175,0.280361,0.335678,-0.570100,-2.261409,-1.432054,0.983043,0.278776,-1.439188
7008,P03496,0.5,Insthoviricetes,-0.315238,-0.325538,-0.597103,-0.780068,-0.737047,0.194158,1.029773,...,-1.525324,-0.391042,-0.984704,-0.512925,-1.280132,-0.553365,-0.333400,0.392162,1.083147,-0.672470
7009,P03496,0.5,Insthoviricetes,-0.347257,1.065222,-0.521005,1.947352,2.418945,-0.919690,-0.564995,...,0.911078,-0.603850,-0.199958,-0.031450,0.072020,-0.486194,-0.152530,-0.004571,-0.719057,0.121415


In [18]:
# Run models on each feature:
feature_pvals = []
for feat in aa_index_feats:
    # Note there didn't appear to be much of an effect from taxa
    selected_data = prepared_data[['is_disordered', 'seq_length', feat + '_avg']]
    model = ols(f"is_disordered ~ seq_length + {feat + '_avg'}", data=selected_data)
    results = model.fit()
    feature_pvals.append({'model_pval': results.f_pvalue, 'rsq': results.rsquared, 'seq_length_p': results.pvalues[1], 'feat_p': results.pvalues[2], 'feat': feat, 'feat_name': aaindex[feat]['description']})

all_results = pd.DataFrame.from_dict(feature_pvals)
all_results.sort_values(by='model_pval', inplace=True)
all_results

,model_pval,rsq,seq_length_p,feat_p,feat,feat_name
282,0.000000e+00,0.263413,1.870999e-186,2.217656e-245,LIFS790102,Conformational preference for parallel beta-st...
317,0.000000e+00,0.271471,5.632210e-187,1.304431e-261,NADH010105,Hydropathy scale based on self-information val...
316,0.000000e+00,0.267692,2.768993e-183,5.603102e-254,NADH010104,Hydropathy scale based on self-information val...
315,0.000000e+00,0.260241,1.273361e-183,4.859173e-239,NADH010103,Hydropathy scale based on self-information val...
314,0.000000e+00,0.238950,3.448364e-186,3.645207e-197,NADH010102,Hydropathy scale based on self-information val...
...,...,...,...,...,...,...
158,1.408426e-208,0.131485,4.441443e-210,4.466652e-01,FUKS010109,Entire chain composition of amino acids in int...
179,1.713502e-208,0.131435,5.151986e-210,6.654279e-01,GEOR030106,Linker propensity from medium dataset (linker ...
161,1.734541e-208,0.131432,5.353802e-210,6.867820e-01,FUKS010112,Entire chain compositino of amino acids in nuc...
68,1.855777e-208,0.131415,7.938184e-210,8.683365e-01,CHAM830105,The number of atoms in the side chain labelled...


In [21]:
full_model = ols(f"is_disordered ~ seq_length + {' + '.join([f + '_avg' for f in aa_index_feats])}", data=prepared_data)
results = full_model.fit()
feat2pval = dict(zip(aa_index_feats, results.pvalues[2:]))
#results.summary()

In [22]:
for feat in sorted(feat2pval.keys(), key=lambda x: feat2pval[x]):
    print(f'{feat}: {feat2pval[feat]}')

QIAN880112: 0.001188715349576864
GEOR030106: 0.0011888317517885938
CHOP780214: 0.0011889650105282735
RADA880102: 0.0011890564601540082
CRAJ730101: 0.0011893917746372239
AVBF000109: 0.0011894734382205758
NAGK730102: 0.0011897450637156498
OOBM770102: 0.001190245611487392
TSAJ990102: 0.001192713355230493
GEIM800109: 0.0011927765088072908
FUKS010101: 0.0011937741239709644
FAUJ830101: 0.0011939120031576925
LIFS790102: 0.0011944055969603247
BEGF750102: 0.0011946207778717518
BLAS910101: 0.0011949620025196713
MEEJ800102: 0.0011998495611930027
VASM830103: 0.0011998807373153778
NADH010105: 0.0012015841085376848
GEIM800102: 0.0012061195770505983
AVBF000101: 0.00121449272709293
WILM950103: 0.0012225018907503229
FUKS010102: 0.0012255132213665114
ROSM880102: 0.0012290164445171547
CHAM830107: 0.0012384942283044768
NADH010106: 0.0012418215085033474
MAXF760102: 0.0012463474645285108
ROSM880104: 0.0012528775149690996
RICJ880112: 0.0012614697689280407
AURR980119: 0.0012670951453592988
OOBM770105: 0.00126